**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://manipulation.csail.mit.edu/clutter.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- define some utility methods/classes that will eventually disappear from this notebook and live in drake.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200918'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py", "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version=version, build=build)
  !pip install pyngrok

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Imports
import numpy as np
from IPython.display import display
from ipywidgets import Textarea

from pydrake.all import ( 
    ConnectMeshcatVisualizer, DiagramBuilder, 
    AddMultibodyPlantSceneGraph, 
    RigidTransform, RotationMatrix
)
from pydrake.all import (Box, CoulombFriction, FindResourceOrThrow, FixedOffsetFrame, GeometryInstance, IllustrationProperties, MeshcatContactVisualizer, Parser, PlanarJoint, ProximityProperties, SpatialInertia)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback


In [ ]:
def contact_force_inspector(slope=0.0, mu=0.1):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.01)

    # Note: adding the geometry directory to the world_body() did not generate contact results.
    ground_instance = plant.AddModelInstance("ground")
    ground_body = plant.AddRigidBody("ground", ground_instance, SpatialInertia())
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("ground"))
    box = Box(10., 10., 10.)
    X_WBox = RigidTransform(RotationMatrix.MakeYRotation(slope), [0, 0, -5.05])
    plant.RegisterCollisionGeometry(ground_body, X_WBox, box, "ground", CoulombFriction(mu, mu))
    plant.RegisterVisualGeometry(ground_body, X_WBox, box, "ground", [.9, .9, .9, 1.0])

    parser = Parser(plant)
    parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/061_foam_brick.sdf"
    ))
    frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))
    plant.AddJoint(PlanarJoint("brick", frame, plant.GetFrameByName("base_link"), damping=[0,0,0]))

#    brick2 = parser.AddModelFromFile(FindResourceOrThrow(
#        "drake/examples/manipulation_station/models/061_foam_brick.sdf"
#    ), "brick2")
#    frame = plant.AddFrame(FixedOffsetFrame("planar_joint_frame2", plant.world_frame(), RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))))
#    plant.AddJoint(PlanarJoint("brick2", frame, plant.GetFrameByName("base_link", brick2), damping=[0,0,0]))


    # Add "ground", with a rotary joint so that we can simulate an incline.
    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(
        builder, 
        scene_graph, 
        zmq_url=zmq_url,
        server_args=server_args)
    meshcat.set_planar_viewpoint(xmin=-.2, xmax=.2, ymin=-.2, ymax=0.3)

    contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
                                           force_threshold=0.0,
                                           contact_force_scale=1.0, 
# needs https://github.com/RobotLocomotion/drake/pull/14131
#                                           radius=0.002,   
                                           plant=plant))
    builder.Connect(scene_graph.get_pose_bundle_output_port(),
                    contact_visualizer.GetInputPort("pose_bundle"))
    builder.Connect(plant.get_contact_results_output_port(),
                    contact_visualizer.GetInputPort("contact_results"))

    diagram = builder.Build()
    meshcat.load()
    context = diagram.CreateDefaultContext()

    textoutput = Textarea(value="", description="contact info: ", layout={'width':'800px','height':'100px'}, style={'description_width':'initial'})
    display(textoutput)
    def my_callback(plant_context):
        results = plant.get_contact_results_output_port().Eval(plant_context)
        textoutput.value = "" if results.num_point_pair_contacts()>0 else "no contact"
        for i in range(results.num_point_pair_contacts()):
            info = results.point_pair_contact_info(i)
            pair = info.point_pair()
            textoutput.value += f"slip speed:{info.slip_speed():.4f}, depth:{pair.depth:.4f}, "
            textoutput.value += "force:" + np.array2string(info.contact_force(), formatter={'float': lambda x: "{:5.2f}".format(x)}) + "\n"

    MakeJointSlidersThatPublishOnCallback(plant, diagram, context, my_callback, resolution=0.001,
                                          lower_limit=[-0.1, -0.1, -np.pi/2.0],# -0.1, -0.1, -np.pi/2.0],
                                          upper_limit=[0.1, 0.1, np.pi/2.0]) #, 0.1, 0.1, np.pi/2.0])    

contact_force_inspector(slope=0.1, mu=1.0)